<a href="https://colab.research.google.com/github/HendricoYehezky/Tugas-1/blob/main/Bab_3_2_Implementasi_Regresi_Linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
### Implementasi regresi linear ###
%matplotlib inline
import random
import torch
from d2l import torch as d2l

In [5]:
### Generating the Dataset ###
def synthetic_data(w, b, num_examples):  
    """Generate y = Xw + b + noise."""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [7]:
### Note that each row in features consists of a 2-dimensional data example and that each row in labels consists of a 1-dimensional label value (a scalar). ###
print('features:', features[0],'\nlabel:', labels[0])

features: tensor([ 0.3045, -1.0844]) 
label: tensor([8.4960])


In [8]:
### By generating a scatter plot using the second feature features[:, 1] and labels, we can clearly observe the linear correlation between the two. ###
d2l.set_figsize()
# The semicolon is for displaying the plot only
d2l.plt.scatter(features[:, (1)].detach().numpy(), labels.detach().numpy(), 1);

ImportError: ignored

<Figure size 252x180 with 1 Axes>

In [9]:
### Reading the Dataset###
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # The examples are read at random, in no particular order
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(
            indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]

In [10]:
### To build some intuition, let us read and print the first small batch of data examples. The shape of the features in each minibatch tells us both the minibatch size and the number of input features. Likewise, our minibatch of labels will have a shape given by batch_size. ###
batch_size = 10

for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    break

tensor([[ 2.0112, -0.6398],
        [ 0.6092, -0.5257],
        [ 0.1573,  1.5935],
        [-0.2205,  0.5521],
        [ 0.2358,  0.9772],
        [-1.0631,  0.3032],
        [ 1.2958,  0.0646],
        [ 1.0031, -0.7301],
        [ 0.8897, -2.0761],
        [-1.0018, -0.1425]]) 
 tensor([[10.3970],
        [ 7.1985],
        [-0.9004],
        [ 1.8741],
        [ 1.3489],
        [ 1.0452],
        [ 6.5909],
        [ 8.7153],
        [13.0383],
        [ 2.6788]])


In [11]:
### Inisialisasi Model parameter ###
w = torch.normal(0, 0.01, size=(2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [13]:
### Mengdefinisikan Model ###
def linreg(X, w, b): 
    """The linear regression model."""
    return torch.matmul(X, w) + b

In [15]:
### Mengdefinsikan Fungsi Loss ###
def squared_loss(y_hat, y):  
    """Squared loss."""
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

In [17]:
### Mengdefinisikan Algoritma Optimisasi ###
def sgd(params, lr, batch_size):  
    """Minibatch stochastic gradient descent."""
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

In [18]:
### Training Data ###
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)  # Minibatch loss in `X` and `y`
        # Compute gradient on `l` with respect to [`w`, `b`]
        l.sum().backward()
        sgd([w, b], lr, batch_size)  # Update parameters using their gradient
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

epoch 1, loss 0.022956
epoch 2, loss 0.000086
epoch 3, loss 0.000053


In [20]:
### In this case, because we synthesized the dataset ourselves, we know precisely what the true parameters are. Thus, we can evaluate our success in training by comparing the true parameters with those that we learned through our training loop. Indeed they turn out to be very close to each other. ###
print(f'error in estimating w: {true_w - w.reshape(true_w.shape)}')
print(f'error in estimating b: {true_b - b}')

error in estimating w: tensor([ 0.0004, -0.0002], grad_fn=<SubBackward0>)
error in estimating b: tensor([0.0014], grad_fn=<RsubBackward1>)
